<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=130774a07b4c812ccd9bde985b579e806aca23db88741efc8ac5fef1c6ea0268
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-11 09:16:24
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.41 C
-------------------
Today PnL: 13.21 K (0.09%)
Current PnL: -17.79 L (-11.97%)
CY Booked + Current PnL: -6.76 L (-4.55%)
-------------------
Total profit:  2.01 L
Total loss:  -19.80 L
-------------------
Total Booked + Current PnL: 20.22 L (16.62%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.8%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.44 L (54.75%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-12.87,63.0,H-LC,11.23,174185.0,18008.0,8082.0,-0.15,11.53,4.64,16.71,22.0,2.23,1.34,30.98,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,59.0,M-LC,4.27,217525.0,16046.0,11159.0,0.32,7.96,5.13,13.51,60.0,1.44,1.67,21.56,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,105.76,63.0,M-SC,5.26,88687.0,-12090.0,12186.0,0.06,-12.00,13.74,0.09,245.0,-0.99,0.68,16.52,OX40N,NTT,DURABLES
41,ITC,452.00,-37.50,55.0,H-LC,3.03,202422.0,2284.0,18603.0,0.22,1.14,9.19,10.44,4.0,0.12,1.55,6.77,X40,NTT,FMCG
49,NATIONALUM,247.44,-43.66,68.0,H-MC,3.38,111698.0,11194.0,19447.0,0.78,11.14,17.41,30.49,79.0,0.58,0.86,47.15,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.14,60.0,M-MC,6.96,222999.0,-1963.0,169613.0,-0.17,-0.87,76.06,74.53,192.0,-0.01,1.71,31.10,XY24,BTT,STEEL
4,ANGELONE,3033.00,7.65,39.0,H-SC,6.92,190814.0,-192.0,57893.0,-0.41,-0.10,30.34,30.21,157.0,-0.00,1.46,20.76,X40N,NTT,FINANCE
54,RAJOOENG,143.33,-32.03,49.0,H-SC,10.61,103000.0,500.0,40335.0,3.77,0.49,39.16,39.83,114.0,0.01,0.79,7.66,AR,ATH,MISC
35,ICICIPRULI,790.00,-20.77,42.0,H-MC,1.97,136267.0,479.0,42270.0,0.12,0.35,31.02,31.48,107.0,0.01,1.05,12.51,X40,ATH,INSURANCE
34,ICICIGI,2260.25,-20.34,44.0,H-MC,3.23,137041.0,908.0,30218.0,-0.18,0.67,22.05,22.86,91.0,0.03,1.05,13.36,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.68,40.0,H-LC,6.30,303688.0,-12154.0,124816.0,0.04,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.31,44.0,H-LC,13.63,218190.0,784.0,52715.0,0.53,0.36,24.16,24.61,48.0,0.01,1.67,55.38,XY24,BTT,FINANCE
56,RELIANCE,1533.00,-14.41,45.0,H-LC,6.29,214609.0,4243.0,24530.0,-0.09,2.02,11.43,13.68,37.0,0.17,1.65,18.87,XY25,NTT,REFINERIES
80,UNITDSPR,1644.00,-14.09,47.0,H-LC,8.16,172933.0,-8826.0,44081.0,0.51,-4.86,25.49,19.39,86.0,-0.20,1.33,2.90,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.16,230146.0,-11778.0,75051.0,0.14,-4.87,32.61,26.15,8.0,-0.16,1.76,8.76,X40,ATH,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.05,58.0,H-LC,1.26,238050.0,-8101.0,87317.0,-0.32,-3.29,36.68,32.18,16.0,-0.09,1.83,31.57,X200,ATH,IT
41,ITC,452.0,-37.50,55.0,H-LC,3.03,202422.0,2284.0,18603.0,0.22,1.14,9.19,10.44,4.0,0.12,1.55,6.77,X40,NTT,FMCG
50,NESTLEIND,1377.0,-7.59,62.0,H-LC,3.33,284521.0,19095.0,37699.0,-0.13,7.19,13.25,21.40,11.0,0.51,2.18,15.54,XY25,NTT,FMCG
20,CIPLA,1795.0,-19.78,53.0,H-LC,5.95,214241.0,9741.0,31665.0,0.79,4.76,14.78,20.25,10.0,0.31,1.64,13.91,X40N,BTT,PHARMA
56,RELIANCE,1533.0,-14.41,45.0,H-LC,6.29,214609.0,4243.0,24530.0,-0.09,2.02,11.43,13.68,37.0,0.17,1.65,18.87,XY25,NTT,REFINERIES


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJOOENG,143.33,-32.03,49.0,H-SC,10.61,103000.0,500.0,40335.0,3.77,0.49,39.16,39.83,114.0,0.01,0.79,7.66,AR,ATH,MISC
68,STARHEALTH,761.00,8.28,51.0,H-SC,3.62,160936.0,-39280.0,115311.0,1.04,-19.62,71.65,37.97,144.0,-0.34,1.23,29.22,XY24,NTT,INSURANCE
27,GREENPANEL,537.00,201.03,53.0,M-SC,6.92,134444.0,-48634.0,127614.0,0.99,-26.56,94.92,43.14,230.0,-0.38,1.03,26.46,XY24,NTT,MISC
33,HONAUT,58357.33,-21.89,45.0,H-SC,2.89,110040.0,-17898.0,65034.0,0.98,-13.99,59.10,36.84,143.0,-0.28,0.84,12.65,X40N,ATH,ELECTRICAL
66,SIS,528.00,2143.01,41.0,H-SC,5.32,60350.0,-20740.0,29409.0,0.82,-25.58,48.73,10.69,156.0,-0.71,0.46,22.31,OX40N,NTT,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,104.20,66.0,H-SC,12.71,128682.0,-13797.0,33316.0,-1.43,-9.68,25.89,13.70,163.0,-0.41,0.99,51.88,OX40N,NTT,BANKS
86,WIPRO,420.0,-10.62,55.0,M-LC,6.27,156854.0,5909.0,103539.0,-1.08,3.91,66.01,72.51,53.0,0.06,1.20,10.17,XR,NTT,IT
39,INFY,2275.0,-17.09,54.0,H-LC,9.78,322993.0,10105.0,161593.0,-1.06,3.23,50.03,54.87,3.0,0.06,2.48,10.09,X40,BTT,IT
63,SFL,1287.0,44.58,62.0,M-SC,8.76,190920.0,-71298.0,141128.0,-1.04,-27.19,73.92,26.63,229.0,-0.51,1.46,18.70,XY24,NTT,MISC
73,TATAMOTORS,1065.0,-18.87,60.0,M-LC,2.27,257403.0,-19120.0,129191.0,-0.90,-6.91,50.19,39.81,54.0,-0.15,1.97,23.36,XY24,NTT,AUTO


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,104.20,66.0,H-SC,12.71,128682.0,-13797.0,33316.0,-1.43,-9.68,25.89,13.70,163.0,-0.41,0.99,51.88,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,105.76,63.0,M-SC,5.26,88687.0,-12090.0,12186.0,0.06,-12.00,13.74,0.09,245.0,-0.99,0.68,16.52,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,56.0,H-SC,2.30,221058.0,-48609.0,84953.0,0.54,-18.03,38.43,13.47,138.0,-0.57,1.70,11.81,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,114.29,49.0,M-SC,2.77,140666.0,-33893.0,33929.0,0.25,-19.42,24.12,0.02,221.0,-1.00,1.08,22.35,OX40N,NTT,PAINTS
19,CERA,9475.0,-19.57,48.0,H-SC,2.27,114732.0,-29071.0,55817.0,0.46,-20.22,48.65,18.60,149.0,-0.52,0.88,26.61,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,53.0,H-LC,13.70,293393.0,-52563.0,176417.0,0.36,-15.19,60.13,35.80,1.0,-0.30,2.25,3.94,X40,BTT,IT
39,INFY,2275.00,-17.09,54.0,H-LC,9.78,322993.0,10105.0,161593.0,-1.06,3.23,50.03,54.87,3.0,0.06,2.48,10.09,X40,BTT,IT
41,ITC,452.00,-37.50,55.0,H-LC,3.03,202422.0,2284.0,18603.0,0.22,1.14,9.19,10.44,4.0,0.12,1.55,6.77,X40,NTT,FMCG
83,VBL,671.64,-15.68,40.0,H-LC,6.30,303688.0,-12154.0,124816.0,0.04,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG
1,ABB,7934.00,-40.64,49.0,H-LC,12.32,248146.0,-13473.0,132684.0,0.15,-5.15,53.47,45.57,7.0,-0.10,1.90,5.59,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.74,188830.0,24160.0,43809.0,-0.07,14.67,23.20,41.28,89.0,0.55,1.45,23.83,X40N,ATH,AC
84,VOLTAS,1918.49,-0.66,54.0,H-MC,3.17,208560.0,16818.0,79211.0,-0.32,8.77,37.98,50.08,99.0,0.21,1.60,16.16,XY25,ATH,AC
24,DABUR,735.00,-1.85,63.0,H-MC,5.60,213525.0,16497.0,73132.0,0.60,8.37,34.25,45.49,102.0,0.23,1.64,22.51,XY24,BTT,FMCG
18,CAMS,5211.76,-3.83,55.0,H-SC,4.74,109368.0,7364.0,36562.0,0.38,7.22,33.43,43.06,122.0,0.20,0.84,25.57,X40N,ATH,MISC
86,WIPRO,420.00,-10.62,55.0,M-LC,6.27,156854.0,5909.0,103539.0,-1.08,3.91,66.01,72.51,53.0,0.06,1.20,10.17,XR,NTT,IT


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-43.66,68.0,H-MC,3.38,111698.0,11194.0,19447.0,0.78,11.14,17.41,30.49,79.0,0.58,0.86,47.15,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.02,64.0,M-SC,1.82,102960.0,11462.0,67295.0,0.22,12.53,65.36,86.07,223.0,0.17,0.79,49.52,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.91,51.0,H-SC,12.12,137122.0,13786.0,119982.0,0.42,11.18,87.50,108.45,119.0,0.11,1.05,36.03,AR,ATH,MISC
86,WIPRO,420.00,-10.62,55.0,M-LC,6.27,156854.0,5909.0,103539.0,-1.08,3.91,66.01,72.51,53.0,0.06,1.20,10.17,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,63.0,H-MC,5.50,181100.0,1292.0,111123.0,0.22,0.72,61.36,62.52,88.0,0.01,1.39,35.05,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6711.11,51.0,L-SC,5.96,74702.0,-19108.0,95723.0,0.03,-20.37,128.14,81.67,269.0,-0.20,0.57,44.66,XR,NTT,CERAMICS
22,COFFEEDAY,80.00,-39.64,58.0,L-SC,43.46,89126.0,-24423.0,64474.0,-0.49,-21.51,72.34,35.27,268.0,-0.38,0.68,118.14,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.00,47.0,H-SC,14.34,89397.0,-11566.0,106096.0,0.49,-11.46,118.68,93.63,148.0,-0.11,0.69,33.15,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,55.0,H-SC,13.05,94380.0,-3600.0,25199.0,0.06,-3.67,26.70,22.05,152.0,-0.14,0.72,37.19,XR,ATH,FINANCE
82,VALIANTORG,838.00,-330.19,58.0,H-SC,8.46,97734.0,-35552.0,120144.0,-0.38,-26.67,122.93,63.47,139.0,-0.30,0.75,63.83,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,58.0,L-SC,43.46,89126.0,-24423.0,64474.0,-0.49,-21.51,72.34,35.27,268.0,-0.38,0.68,118.14,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,64.0,M-SC,3.01,82440.0,-19770.0,113157.0,0.48,-19.34,137.26,91.37,208.0,-0.17,0.63,33.83,XY25,NTT,FINANCE
8,ATULAUTO,844.0,3683.78,61.0,M-SC,7.43,117941.0,-29427.0,81238.0,0.66,-19.97,68.88,35.16,236.0,-0.36,0.90,20.47,XY24,NTT,AUTO
75,TITAGARH,1548.0,-3.20,64.0,H-SC,6.48,158821.0,-33194.0,112080.0,-0.14,-17.29,70.57,41.08,158.0,-0.30,1.22,34.58,XY24,NTT,ENGINEERING
64,SHALBY,327.0,1086.60,74.0,M-SC,1.86,161469.0,-19674.0,65137.0,-0.47,-10.86,40.34,25.10,235.0,-0.30,1.24,29.40,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.29
1,25,44.34
2,50,75.05


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.87
LC    35.39
MC    23.72
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.66
X40      16.05
X40N     13.13
XY25     10.28
XR        9.52
AR        8.39
OX40N     7.75
X200      1.83
MH        1.79
X5K       1.45
SR        1.13
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.83
H-SC    25.05
H-MC    20.64
M-SC    14.17
M-LC     7.45
M-MC     2.75
L-SC     1.65
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.95,-15.37,73.78
FMCG,12.30,-3.42,38.48
FINANCE,9.05,-18.20,60.70
BANKS,7.79,-12.31,59.99
PAINTS,6.15,-15.12,40.89
MISC,6.07,-15.89,82.32
ELECTRICAL,5.83,-8.12,67.71
AC,3.80,2.17,30.97
AUTO,3.47,-15.65,67.80


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2725096.0,22
AR,1176239.0,9
XR,1051114.0,13
X40,873203.0,10
X40N,563935.0,10
OX40N,447507.0,10
XY25,411279.0,7
SR,196923.0,2
X5K,128872.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2757054.0,28
M-SC,1397164.0,17
H-LC,1366343.0,15
H-MC,1152676.0,15
M-LC,406987.0,5
M-MC,319529.0,2
L-SC,244841.0,3
L-MC,59769.0,1
L-LC,39467.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,912988.0,6
M-SC,XY24,815434.0,7
H-SC,AR,643894.0,5
H-LC,X40,582396.0,5
H-SC,XR,502040.0,6
H-MC,XY24,461013.0,4
H-LC,AR,373042.0,2
M-MC,XY24,319529.0,2
H-SC,X40N,246415.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
